See the IREE docs/using_colab.md document for instructions.

This notebook shows off some concepts of the low level IREE python bindings.

In [0]:
import numpy as np
from pyiree import binding


In [0]:
def dump_module(m):
  print("Loaded module:", m.name)
  i = 0
  while True:
    f = m.lookup_function_by_ordinal(i)
    if not f: break
    print("  Export:", f.name, "-> args(", f.signature.argument_count, 
          "), results(", f.signature.result_count, ")")
    i += 1  

In [7]:
# Compile a module.
ctx = binding.compiler.CompilerContext()
input_module = ctx.parse_asm("""
  func @simple_mul(%arg0: tensor<4xf32>, %arg1: tensor<4xf32>) -> tensor<4xf32>
        attributes { iree.module.export } {
      %0 = "xla_hlo.mul"(%arg0, %arg1) {name = "mul.1"} : (tensor<4xf32>, tensor<4xf32>) -> tensor<4xf32>
      return %0 : tensor<4xf32>
  }
  """)
blob = input_module.compile_to_sequencer_blob()
m = binding.vm.create_module_from_blob(blob)
dump_module(m)

Loaded module: module
  Export: simple_mul -> args( 2 ), results( 1 )


In [8]:
# Initialize the runtime and register the module.
# Use the CPU interpreter driver (which has the most implementation done):
driver_name = "interpreter"

# Live on the edge and give the vulkan driver a try:
# driver_name = "vulkan"

policy = binding.rt.Policy()
instance = binding.rt.Instance(driver_name=driver_name)
context = binding.rt.Context(instance=instance, policy=policy)
context.register_module(m)
f = context.resolve_function("module.simple_mul")

print("INVOKE F:", f.name)
arg0 = context.wrap_for_input(np.array([1., 2., 3., 4.], dtype=np.float32))
arg1 = context.wrap_for_input(np.array([4., 5., 6., 7.], dtype=np.float32))

# Invoke the function and wait for completion.
inv = context.invoke(f, policy, [arg0, arg1])
print("Status:", inv.query_status())
inv.await_ready()

# Get the result as a numpy array and print.
results = inv.results
print("Results:", results)
result = results[0].map()
print("Mapped result:", result)
result_ary = np.array(result, copy=False)
print("NP result:", result_ary)


INVOKE F: simple_mul
Status: True
Results: [<pyiree.binding.hal.BufferView object at 0x00000179E51410D8>]
Mapped result: <pyiree.binding.hal.MappedMemory object at 0x00000179E51412D0>
NP result: [ 4. 10. 18. 28.]
